In [52]:
import hax, pax
from pax import core
from hax.runs import is_mc
import pandas as pd
import pickle, io, sys, os
from pax.plugins.io import Pickle

import numpy as np
import zlib
from pax import configuration


print('PAX', pax.__version__)
print('HAX', hax.__version__)


class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        renamed_module = module
        if module == "wfsim.pax_datastructure.datastructure":
            renamed_module = "pax.datastructure"

        return super(RenameUnpickler, self).find_class(renamed_module, name)


def renamed_load(file_obj):
    return RenameUnpickler(file_obj).load()


def renamed_loads(pickled_bytes):
    file_obj = io.BytesIO(pickled_bytes)
    return renamed_load(file_obj)


m = sys.modules['pax.plugins.io.Pickle']
m.pickle.loads = renamed_loads


def get_run_number(run_id):
        return int(run_id)

m = sys.modules['hax.runs']
m.get_run_number = get_run_number


def init_core(run_number):
    core_processor = core.Processor(
        config_names=['_base', 'XENON1T'],
        just_testing=False,
        config_dict={
            'pax':{
                'input_name' : '/dali/lgrandi/zhut/sim/pax_data/XENON1T_MC_%s'%run_number,
                'output_name': '/dali/lgrandi/zhut/sim/pax_data/processed/%s'%run_number,
                'look_for_config_in_runs_db': False},
            'MC':{
                'mc_generated_data': True,
                'mc_sr_parameters': "sr1",
                'mc_run_number': run_number,}
        })
    return core_processor

PAX 6.10.1
HAX 2.5.0


In [69]:
name = '0'
processor = init_core(name)

processor MainProcess L66 INFO This is PAX version 6.10.1, running with configuration for XENON1T.
/cvmfs/xenon.opensciencegrid.org/releases/anaconda/2.4/envs/pax_head/lib/python3.4/site-packages/pax-6.10.1-py3.4.egg/pax/PatternFitter.py:68: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.data *= adjust_to_qe[[np.newaxis] * self.dimensions]
ReadZipped MainProcess L102 INFO InputFromFolder: Selecting file /dali/lgrandi/zhut/sim/pax_data/XENON1T_MC_0/XENON1T-0-000000000-000000999-000001000.zip (number 1/2 in folder) for reading




Output file /dali/lgrandi/zhut/sim/pax_data/processed/0.root already exists, overwriting.


/cvmfs/xenon.opensciencegrid.org/releases/anaconda/2.4/envs/pax_head/lib/python3.4/site-packages/pax-6.10.1-py3.4.egg/pax/PatternFitter.py:68: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.data *= adjust_to_qe[[np.newaxis] * self.dimensions]


In [70]:
dtype = [
('event_number', 'i8'),
('event_start', 'i8'),
('run_number', 'i8'),
('s1', 'f8'),
('s1_area_fraction_top', 'f8'),
('s1_range_50p_area', 'f8'),
('s1_rise_time', 'f8'),
('s1_center_time', 'f8'),
('s1_n_hits', 'f8'),
('s1_tight_coincidence', 'i4'),
('time', 'i8'),
('endtime', 'i8'),
]


def extract_data(result, event, index):
    cnt = 0
    
    for p in event.s1s():
        result[index]['event_number'] = event.event_number
        result[index]['event_start'] = event.start_time
        result[index]['s1'] = p.area
        result[index]['s1_area_fraction_top'] = p.area_fraction_top
        result[index]['s1_range_50p_area'] = p.range_50p_area
        result[index]['s1_rise_time'] = -p.area_decile_from_midpoint[1]
        result[index]['s1_center_time'] = p.center_time
        result[index]['s1_n_hits'] = p.n_hits
        result[index]['time'] = p.left * 10 + event.start_time
        result[index]['endtime'] = p.right * 10 + event.start_time
        result[index]['s1_tight_coincidence'] = p.tight_coincidence

        index += 1
        cnt += 1
        if cnt == 10:
            break
    
    return index


In [71]:
result = np.zeros(11 * 10, dtype=np.dtype(dtype))
index = 0
for event in processor.get_events():

    for j, plugin in enumerate(processor.action_plugins[:-6]):
        if 'PosRec' in plugin.name: continue
        event = plugin.process_event(event)

    index = extract_data(result, event, index)

    if event.event_number == 10:
        break
        
result['run_number'] = int(name)

/cvmfs/xenon.opensciencegrid.org/releases/anaconda/2.4/envs/pax_head/lib/python3.4/site-packages/pax-6.10.1-py3.4.egg/pax/plugins/peak_processing/BasicProperties.py:30: RuntimeWarning: divide by zero encountered in true_divide
  peak.mean_amplitude_to_noise = np.average(hits['height']/hits['noise_sigma'], weights=hits['area'])


In [72]:
import zlib
with open('/dali/lgrandi/zhut/minitrees/pax_s1b_tb/%s_HitPerPE'%name, 'wb') as f:
    result = zlib.compress(result, 4)
    f.write(result)